In [1]:
import numpy as np
import time
from sklearn.model_selection import train_test_split
#from tqdm.autonotebook import tqdm, trange

import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn


BATCH_SIZE = 32
import torch._utils
import torch
import argparse
import torch.nn as nn
import torch.optim as optim
import time
from tqdm.auto import tqdm
# from model import CNNModel
# from datasets import train_loader, valid_loader
# from utils import save_model, save_plots


In [10]:
transform = transforms.Compose([
    transforms.Resize((int(300), int(300))),
    transforms.ToTensor()
])

data = datasets.ImageFolder(
    root=r"C:\Users\vikas\OneDrive\Desktop\Work\Classifier_welding\Welding_Photos",
    transform=transform
)
# train_idx, valid_idx = train_test_split(list(range(len(data))), train_size=0.9)

# dataset =  torch.utils.data.Subset(data)
#     'val': torch.utils.data.Subset(data, valid_idx)


trainloader = torch.utils.data.DataLoader(
        dataset=data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4
    )
#     'val': torch.utils.data.DataLoader(
#         dataset=dataset['val'], batch_size=BATCH_SIZE, shuffle=False, num_workers=4
#     ),


# transform = transforms.Compose([
#     transforms.Resize((int(300), int(300))),
#     transforms.ToTensor()
# ])

# dataset_test = datasets.ImageFolder(
#     root="../input/children-vs-adults-images/test",
#     transform=transform
# )

# dataloader_test = torch.utils.data.DataLoader(dataset_test, shuffle=False, batch_size=BATCH_SIZE)

# datasets_sizes = {x: len(dataset[x]) for x in ['train', 'val']}
class_names = np.array(data.classes)

In [3]:
import torchvision.models as models
import torch._utils
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(in_features=512, out_features=2, bias=True)

C:\Users\vikas\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vikas\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
lr = 1e-3
# epochs = args['epochs']
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")
model = resnet18.to(device)

print(model)
# total parameters and trainable parameters
# total_params = sum(p.numel() for p in model.parameters())
# print(f"{total_params:,} total parameters.")
# total_trainable_params = sum(
#     p.numel() for p in model.parameters() if p.requires_grad)
# print(f"{total_trainable_params:,} training parameters.")
# optimizer
optimizer = optim.Adam(resnet18.parameters(), lr=lr)
# loss function
criterion = nn.CrossEntropyLoss()

Computation device: cuda

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (re

In [11]:
def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc

In [13]:
transform_inf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(300),
    transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.5, 0.5, 0.5],
#         std=[0.5, 0.5, 0.5]
#     )
])  

In [14]:
# lists to keep track of losses and accuracies
epochs = 32
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
# start the training
for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(model, trainloader, 
                                              optimizer, criterion)
#     valid_epoch_loss, valid_epoch_acc = validate(model, valid_loader,  
#                                                  criterion)
    train_loss.append(train_epoch_loss)
#     valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
#     valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
#     print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)
    time.sleep(5)
    
# save the trained model weights
save_model(epochs, model, optimizer, criterion)
# save the loss and accuracy plots
# save_plots(train_acc, valid_acc, train_loss, valid_loss)
print('TRAINING COMPLETE')

[INFO]: Epoch 1 of 32
Training


  0%|          | 0/24 [00:00<?, ?it/s]

Training loss: 0.094, training acc: 95.828
--------------------------------------------------
[INFO]: Epoch 2 of 32
Training


  0%|          | 0/24 [00:00<?, ?it/s]

Training loss: 0.008, training acc: 99.865
--------------------------------------------------
[INFO]: Epoch 3 of 32
Training


...

Training loss: 0.002, training acc: 100.000
--------------------------------------------------
[INFO]: Epoch 4 of 32
Training


  0%|          | 0/24 [00:00<?, ?it/s]

Training loss: 0.000, training acc: 100.000
--------------------------------------------------
[INFO]: Epoch 5 of 32
Training


  0%|          | 0/24 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
torch.save(model.state_dict(), 'classifier_01.pth')

# Inference


In [27]:
import torchvision.models as models
import torch._utils
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(in_features=512, out_features=2, bias=True)
model_loaded = resnet18
model_loaded.load_state_dict(torch.load('classifier_01.pth'))
model_loaded.eval()
model_loaded.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
# the computation device
device = ('cuda' if torch.cuda.is_available() else 'cpu')
# list containing all the class labels
labels = [
    'good', 'bad', 
    ]

In [30]:
import cv2
image = cv2.imread(r"C:\Users\vikas\OneDrive\Desktop\Work\Classifier_welding\Welding_Photos\Good_weld\70.png")
# get the ground truth class
# gt_class = args['input'].split('/')[-2]
orig_image = image.copy()
# convert to RGB format
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = transform_inf(image)
# add batch dimension
image = torch.unsqueeze(image, 0)
with torch.no_grad():
    outputs = model_loaded(image.to(device))
output_label = torch.topk(outputs,1)
pred_class = labels[int(output_label.indices)]

cv2.putText(orig_image, 
    f"Pred: {pred_class}",
    (10, 55),
    cv2.FONT_HERSHEY_SIMPLEX, 
    0.6, (0, 0, 255), 2, cv2.LINE_AA
)
print(f"GT: 'ground truth', pred: {pred_class}")
cv2.imshow('Result', orig_image)
cv2.waitKey(0)
# output_label

GT: 'ground truth', pred: good


-1

In [18]:
output_label

torch.return_types.topk(
values=tensor([[3.1378]], device='cuda:0'),
indices=tensor([[0]], device='cuda:0'))